#  🤗 Transformers

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://huggingface.co/docs/transformers/index
* https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline.task
* https://huggingface.co/docs/transformers/preprocessing
* https://huggingface.co/blog/getting-started-with-embeddings
* https://huggingface.co/sentence-transformers
* https://habr.com/ru/articles/704592/

https://huggingface.co/docs/transformers/training
https://huggingface.co/docs/datasets/main/en/repository_structure
https://huggingface.co/docs/datasets/main/en/package_reference/loading_methods#datasets.load_dataset
https://huggingface.co/docs/transformers/v4.35.2/en/training#prepare-a-dataset
https://huggingface.co/docs/datasets/process
https://huggingface.co/docs/evaluate/index
https://huggingface.co/docs/transformers/main_classes/trainer
https://huggingface.co/docs/transformers/v4.35.2/en/main_classes/trainer#transformers.TrainingArguments

## Задачи для совместного разбора

In [ ]:
!pip install transformers

1\. Обсудите основные возможности и экосистему пакета 🤗 Transformers на примере задачи поиска ответа на вопрос в тексте.

In [ ]:
context = """The seminars on Deep Learning and Natural Language Processing were truly captivating,
providing a deep dive into the intricacies of these disciplines.
The wealth of knowledge and insights gained during the sessions was commendable.
However, it's disheartening to note the scarcity of homework assignments.
Anastasia, in particular, is quite concerned that the limited number of assignments might
fall short of even reaching 30. While the seminars were intellectually stimulating,
the desire for more hands-on practice through assignments remains strong,
as it is crucial for reinforcing the theoretical understanding acquired during the classes."""

In [ ]:
question1 = "What would be the ideal number of homework assignments for Anastasia"
question2 = "What are the shortcomings of the course?"

In [ ]:
from transformers import pipeline

question_answerer = pipeline(
    "question-answering",
    model='distilbert-base-cased-distilled-squad'
)

result = question_answerer(
    question=question1,
    context=context
)
result

In [ ]:
context[result["start"]: result["end"]]

'30'

In [ ]:
result["answer"]

'30'

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
# from transformers import AutoTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased-distilled-squad')
model = DistilBertModel.from_pretrained('distilbert-base-cased-distilled-squad')
inputs = tokenizer(question1, context, return_tensors="pt")

In [ ]:
inputs

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[ 1.2595, -0.6063,  1.6427,  ..., -0.4271,  0.0788, -0.3389],
         [ 0.7698, -0.7574,  1.5950,  ..., -1.1511, -0.4955, -0.1044],
         [ 1.1409, -0.8310,  1.4844,  ..., -1.0516, -0.3719,  0.0044],
         ...,
         [ 1.0212, -0.6903,  1.8762,  ...,  0.0586,  0.0653, -0.5465],
         [ 1.3074, -0.4359,  1.9121,  ..., -0.4082,  0.2553, -0.8889],
         [ 0.8205, -1.0346,  1.5188,  ..., -0.9568,  1.0282, -0.2448]]]), hidden_states=None, attentions=None)


2\. Обсудите основные шаги по дообучению моделей из экосистемы 🤗 Transformers.

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Загрузите данные из файла `reviews_polarity.csv`. Среди предобученных моделей найдите модель для классификации тональности русскоязычного текста (позитивный/негативный или позитивный/негативный/нейтральный). Протестируйте данную модель на нескольких предложениях, используя `transformers.pipeline`. Выведите результаты работы в следующем виде:

```
sentence1 -> class1
sentence2 -> class2
...
```

Получите прогноз для всех текстов из файла и посчитайте F1-score. Для ускорения работы модели вы можете перенести ее на GPU и использовать возможности pipeline для работы с батчами и генераторами.

- [ ] Проверено на семинаре

In [1]:
import pandas as pd
from sklearn.metrics import f1_score
from transformers import pipeline


In [2]:
from tqdm import tqdm

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
data = pd.read_csv('reviews_polarity.csv')
data

,text,class
0,"Очень хорошо что открылась 5 ка, теперь не над...",1
1,"Тесно, вечная сутолока, между рядами трудно ра...",0
2,Магазин в пешей доступности. После ремонта и р...,1
3,Магазин хороший цены и скидки нормальные токо ...,1
4,Сложно найти в торговом центре. А магазин - норм),1
...,...,...
38213,Магазин очень хороший есть большой ассортимент...,1
38214,"Удобный, но маленький и ещё не обновили как др...",1
38215,Очень хочется пожелать этому магазину стать та...,0
38216,"Нравится ваш магазин, персонал одекватный, пор...",1


In [9]:
data['class'].unique()

array([1, 0])

In [5]:
import pandas as pd
from sklearn.metrics import f1_score
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

In [25]:
classifier = pipeline('sentiment-analysis', model='blanchefort/rubert-base-cased-sentiment', device=0, batch_size=8)


In [26]:
sentences = [
    "Это замечательный день!",
    "Ужасный опыт, не рекомендую.",
    "Фильм понравился, рекомендую всем посмотреть."
]

for sentence in sentences:
    result = classifier(sentence)
    print(f"{sentence} -> {result[0]['label']}")


Это замечательный день! -> POSITIVE
Ужасный опыт, не рекомендую. -> NEGATIVE
Фильм понравился, рекомендую всем посмотреть. -> POSITIVE


In [31]:
predictions = classifier(data['text'].tolist())

In [42]:
predictions[:5]

[{'label': 'POSITIVE', 'score': 0.9645202159881592},
 {'label': 'NEGATIVE', 'score': 0.7516101598739624},
 {'label': 'POSITIVE', 'score': 0.9681913256645203},
 {'label': 'POSITIVE', 'score': 0.9519004821777344},
 {'label': 'POSITIVE', 'score': 0.9813418388366699}]

In [45]:
def f(label):
    return 1 if label == 'POSITIVE' else 0

In [48]:
labels = [f(pred['label']) for pred in predictions]

In [49]:
labels[:5]

[1, 0, 1, 1, 1]

In [50]:
f1 = f1_score(data['class'], labels)
f1

0.814415985743265

<p class="task" id="2"></p>

2\. Создайте токенизатор, соответствующий модели из предыдущего задания, используя класс `AutoTokenizer`. Возьмите одно предложение из набора данных и токенизируйте его, используя созданный объект. Выведите на экран полученный результат. Выполните обратное преобразование: получите текст по набору токенов и выведите результат на экран.

Получите батч из 10 предложений и токенизируйте его. Продемонстрируйте возможности токенизатора для паддинга, обрезки, преобразования в тензоры.
Решите задачу 1, создав объект токенизатора (`AutoTokenizer`) и модель (`AutoModelForSequenceClassification`).

- [ ] Проверено на семинаре

In [51]:
model_name = "blanchefort/rubert-base-cased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sample_sentence = data['text'].iloc[0]

tokens = tokenizer.tokenize(sample_sentence)
tokens

['Очень',
 'хорошо',
 'что',
 'открылась',
 '5',
 'ка',
 ',',
 'теперь',
 'не',
 'надо',
 'далеко',
 'ехать',
 'все',
 'рядом',
 '!']

In [52]:
decoded_sentence = tokenizer.decode(tokenizer.encode(sample_sentence))
decoded_sentence

'[CLS] Очень хорошо что открылась 5 ка, теперь не надо далеко ехать все рядом! [SEP]'

In [53]:
batch_sentences = data['text'][:10].tolist()
batch_tokens = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
batch_tokens

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[  101,  3065,  1643,   825, 33736,   146,   871,   128,  1661,   802,
          1199,  3194,  5857,   888,  3504,   106,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101, 10229,   793,   128, 48717,  7607,  4548,   813,   128,  1862,
         56278,  4703, 76683,   128, 53506,   132, 12213,  2497,   893,   132,
          4626,   802,  9458,   132,   102,     0,     0,     0,     0],
        [  101, 62127,   340,  6836,   371, 36716,   132,  2525, 17375,   322,
         22263,  2774,  5467,  1554,   322, 15530,  1229,   132, 67740,  2892,
         17311, 63879,   909,   322, 94994,   132,   102,     0,     0],
        [  101, 62127,  4445,  3877,   322, 28892,  9639, 77737,   888,  1210,
           352,  1312,  2587,  1980, 42170, 21867,   802,   352,  1437,   999,
          1928,   801, 40458,  1583,   102,     0,     0,     0,     0],
        [  101, 26093,  2692,   340, 51754,  6339,   132,   44

In [54]:
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0, batch_size=16)

In [55]:
sentences = [
    "Это замечательный день!",
    "Ужасный опыт, не рекомендую.",
    "Фильм понравился, рекомендую всем посмотреть."
]

for sentence in sentences:
    result = classifier(sentence)
    print(f"{sentence} -> {result[0]['label']}")


Это замечательный день! -> POSITIVE
Ужасный опыт, не рекомендую. -> NEGATIVE
Фильм понравился, рекомендую всем посмотреть. -> POSITIVE


In [56]:
predictions = classifier(data['text'].tolist())

In [57]:
def f(label):
    return 1 if label == 'POSITIVE' else 0

In [58]:
labels = [f(pred['label']) for pred in predictions]

In [59]:
f1 = f1_score(data['class'], labels)
f1

0.814415985743265

<p class="task" id="3"></p>

3\. Разбейте данные из файла `reviews_polarity.csv` на обучающее и валидационное множество в соотношении 80 на 20. Создайте папку `reviews_polarity_dataset` и сохраните в нее полученные фрагменты данных под названием `train.csv` и `test.csv`. Создайте объект `datasets.Dataset`, используя функцию `load_dataset`.

Токенизируйте строки при помощи токенизатора, соотвествующего модели `rubert-base-cased-sentiment`. Удалите из датасета поле `text` после токенизации, замените поле `class` на `labels` и приведите данные к тензорам `torch`.

Создайте два `DataLoader` на основе обучающего и валидационного множества. Получите батч из обучающего множества и выведите его на экран.

- [ ] Проверено на семинаре

In [ ]:
pip install datasets

In [80]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset

In [144]:
tokenizer = AutoTokenizer.from_pretrained("blanchefort/rubert-base-cased-sentiment")

In [145]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [146]:
train_data.to_csv('/content/reviews_polarity_dataset/train.csv')
test_data.to_csv('/content/reviews_polarity_dataset/test.csv')

In [163]:
data_files = {'train': 'train.csv', 'test': 'test.csv'}
dataset = load_dataset('reviews_polarity_dataset', data_files=data_files)

In [164]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'class'],
        num_rows: 30574
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'class'],
        num_rows: 7644
    })
})

In [165]:
dataset = dataset.remove_columns('Unnamed: 0')

In [166]:
def tokenize_func(examples):
    return tokenizer(examples, return_tensors='pt', padding=True)

In [167]:
model_name = "blanchefort/rubert-base-cased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [169]:
dataset = dataset.map(
    tokenize_func,
    input_columns=['text'],
    batched=True
)

Map:   0%|          | 0/7644 [00:00<?, ? examples/s]

In [170]:
dataset = dataset.rename_column("class", "labels")

In [171]:
dataset = dataset.remove_columns('text')

In [172]:
dataset.set_format(type='torch')

In [173]:
train_loader = DataLoader(dataset['train'], batch_size=32)
test_loader = DataLoader(dataset['test'], batch_size=32)

In [174]:
next(iter(train_loader))

{'labels': tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
         0, 0, 1, 1, 1, 1, 1, 1]),
 'input_ids': tensor([[  101, 14220,  1297,  ...,     0,     0,     0],
         [  101,  3065, 31617,  ...,     0,     0,     0],
         [  101, 82963,  5187,  ...,     0,     0,     0],
         ...,
         [  101, 13893,  6419,  ...,     0,     0,     0],
         [  101,  3065,  6467,  ...,     0,     0,     0],
         [  101, 62127,  4445,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

<p class="task" id="4"></p>

4\. Создайте модель при помощи класса `AutoModelForSequenceClassification`, заменив голову модели в соответствии с задачей бинарной классификации. Используя стандартный цикл обучения `torch`, настройте модель для решения задачи бинарной классификации. Во время обучения выводите на экран значение функции потерь (используйте готовые значения, которые генерирует модель) на обучающем множестве и f1 на валидационном множестве.

Здесь и далее для ускорения процесса обучения вы можете заморозить часть сети или уменьшить размер наборов данных, выбрав небольшое подмножество примеров.

- [ ] Проверено на семинаре

In [175]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
model_name = "blanchefort/rubert-base-cased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels=2)

In [193]:
model.classifier = nn.Linear(in_features = model.classifier.in_features, out_features=2, bias=True)

In [194]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

In [227]:
import numpy as np

In [229]:
epochs = 5

In [230]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

<p class="task" id="5"></p>

5\. Создайте модель при помощи класса `AutoModelForSequenceClassification`, заменив голову модели в соответствии с задачей бинарной классификации. Используя `transformers.Trainer`, настройте модель для решения задачи бинарной классификации. При настройке `Trainer` укажите количество эпох, равное 5. Во время обучения выводите на экран значение функции потерь на обучающем множестве и f1 на валидационном множестве.  

- [ ] Проверено на семинаре


## Обратная связь
- [ ] Хочу получить обратную связь по решению